In [1]:
import pandas as pd
import os
from supabase import create_client, Client

In [ ]:
make = 'Ford'
make.lower()

In [5]:
selected_make = f'{make.lower()}_table'

In [6]:
selected_make

'ford_table'

### Vehicle Security System Data Structure

#### Overview
The security system information varies significantly between manufacturers, requiring separate database tables to efficiently store and manage the data.

1. Ford Motor Company Group
Brands: Ford, Lincoln, Mercury, Mazda

Key Features:
- PATS (Passive Anti-Theft System) specific information
- Parameter reset requirements
- Module location details
- Split between keyed and push-to-start variants

2. General Motors Group
Brands: Chevrolet, Cadillac, Buick, GMC, Pontiac, Saturn, Hummer

Key Features:
- Hardware part numbers
- Cloning compatibility information
- Multiple security system types (VATS, PK3, etc.)

3. Basic Security Info Group
Brands: Toyota, Honda, Nissan, etc.

Key Features:
- Standard security system information
- No special procedures or additional data required

In [2]:
# Read the DataFrame and save it in the variable df.
df = pd.read_excel('C:\Language_Projects\Language_Projects\Python\Flagship_1\Immo_Assistant.app\data\year_make_model_df.xlsx')

<>:2: SyntaxWarning: invalid escape sequence '\L'
<>:2: SyntaxWarning: invalid escape sequence '\L'
C:\Users\Weverson Barbieri\AppData\Local\Temp\ipykernel_8552\507487594.py:2: SyntaxWarning: invalid escape sequence '\L'
  df = pd.read_excel('C:\Language_Projects\Language_Projects\Python\Flagship_1\Immo_Assistant.app\data\year_make_model_df.xlsx')


In [3]:
# Get a list with all column names
df.columns

Index(['Year', 'Make', 'Model', 'Security', 'ParameterReset'], dtype='object')

In [4]:
# Create a map to rename the column names before import the csv file to Supabase
df_column_names_map = {
                    'Year': 'year',
                    'Make': 'make',
                    'Model': 'model',
                    'Security': 'security',
                    'ParameterReset': 'parameter_reset'
}

In [5]:
# Create a function to rename the columns
def rename_column_names(df):
    df_renamed_columns = df.rename(columns=df_column_names_map)
    return df_renamed_columns

In [6]:
# Call the function to rename the columns
df_renamed_columns = rename_column_names(df)

In [7]:
# Print the head to confirm the changes
df_renamed_columns.head(2)

,year,make,model,security,parameter_reset
0,1986,Chevrolet,Corvette,VATS,NaN
1,1987,Chevrolet,Corvette,VATS,NaN


### Data Export and Supabase Import Process
After renaming the columns, the data will be exported locally to the project's root folder and then imported into the Supabase cloud database. This will store the table data before any manipulation or manufacturer-specific division.

In [8]:
# Download the df to csv file
df_renamed_columns.to_csv('C:\Language_Projects\Language_Projects\Python\Flagship_1\Immo_Assistant.app\dataframe_csv_files\df_main.csv', index=False)

<>:2: SyntaxWarning: invalid escape sequence '\L'
<>:2: SyntaxWarning: invalid escape sequence '\L'
C:\Users\Weverson Barbieri\AppData\Local\Temp\ipykernel_17444\3442244793.py:2: SyntaxWarning: invalid escape sequence '\L'
  df_renamed_columns.to_csv('C:\Language_Projects\Language_Projects\Python\Flagship_1\Immo_Assistant.app\dataframe_csv_files\df_main.csv', index=False)


In [8]:
# Get Supabase URL from environment variable
url: str = os.environ.get("NEXT_PUBLIC_SUPABASE_URL")
# Get Supabase anonymous key from environment variable 
key: str = os.environ.get("NEXT_PUBLIC_SUPABASE_ANON_KEY")

In [9]:
# Initialize Supabase client with URL and anonymous key
Supabase: Client = create_client(url, key)

In [10]:
def query_table_from_supabaseto_df(table_name):
    # Query all records from the table in Supabase and store the response 
    response_table = (Supabase.table(table_name).select("*").execute())

    # Store the queried data from Supabase response into a variab
    year_make_model_data = response_table.data

    # Convert the data to dataframe format
    df = pd.DataFrame(year_make_model_data)

    # Drop unnecessary columns came from the Supabase table
    df_dropped_columns = df.drop(columns=['id', 'created_at', 'updated_at'])

    # Return a clear df after query
    return df_dropped_columns

In [11]:
# Call the function to get a cleared df after query from Supabase
df_dropped_columns = query_table_from_supabaseto_df("year_make_model_table")

In [12]:
# Check which models the df has and print only the unique values
df_dropped_columns['make'].value_counts()

make
BMW           98
Toyota        80
Ford          65
Acura         65
Honda         63
Dodge         57
Chrysler      48
Chevrolet     47
Mazda         45
Lexus         41
Oldsmobile    39
Nissan        38
GMC           36
Cadillac      34
Buick         28
Pontiac       27
Mercury       24
Audi          23
Infiniti      21
Lincoln       21
Jeep          20
Jaguar        19
Mitsubishi    19
Volkswagen    18
Plymouth      15
Saturn         7
Land Rover     1
Mini           1
Name: count, dtype: int64

In [13]:
# Confirm if the Dtype from the column Model changed to string.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3463 entries, 0 to 3462
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Year            3463 non-null   int64 
 1   Make            3463 non-null   object
 2   Model           3463 non-null   object
 3   Security        3156 non-null   object
 4   ParameterReset  938 non-null    object
dtypes: int64(1), object(4)
memory usage: 135.4+ KB


In [14]:
# Create a new dataframe containing only Chevrolet: 'Chevrolet', 'Pontiac', 'Cadillac', 'Buick', 'Oldsmobile', 'GMC', 'Saturn', 'Hummer'
chevrolet_list = ['Chevrolet', 'Pontiac', 'Cadillac', 'Buick', 'Oldsmobile', 'GMC', 'Saturn', 'Hummer']

In [15]:
# Create a dataframe containing only chevrolet brands 
df_chevrolet = df_dropped_columns[df_dropped_columns['make'].isin(chevrolet_list)]

In [16]:
# Drop the column 'ParameterReset' as it is not applicable to Chevrolet
df_chevrolet_dropped_column = df_chevrolet.drop(columns=['parameter_reset'])

#### After creating the Chevrolet brands DataFrame, hardware part numbers must be manually integrated from proprietary company systems. This multi-step process ensures data accuracy and security compliance.

Process Steps
    Data Preparation

1. Export filtered Chevrolet DataFrame to Excel
    - Create template for hardware number entry
    - Manual Integration

2. Access company's internal systems
    - Add hardware numbers from GM compatibility charts
    - Cross-reference with existing documentation
    - Quality Control

3. Verify all entered part numbers
    - Validate data format consistency
    - Cross-check cloning compatibility status
    
Note: Manual integration is required due to system access restrictions and data sensitivity requirements.

In [17]:
# After including the hardware part numbers, read the df with all chevrolet brands to make the final changes before import to Supabase
df_chevrolet_xlsx = pd.read_excel("C:\Language_Projects\Language_Projects\Python\Flagship_1\Immo_Assistant.app\dataframe_each_make_xlsx_file\chevrolet_hardware_list.xlsx")

<>:2: SyntaxWarning: invalid escape sequence '\L'
<>:2: SyntaxWarning: invalid escape sequence '\L'
C:\Users\Weverson Barbieri\AppData\Local\Temp\ipykernel_8552\3614794800.py:2: SyntaxWarning: invalid escape sequence '\L'
  df_chevrolet_xlsx = pd.read_excel("C:\Language_Projects\Language_Projects\Python\Flagship_1\Immo_Assistant.app\dataframe_each_make_xlsx_file\chevrolet_hardware_list.xlsx")


In [18]:
# Print the columns
df_chevrolet_xlsx.columns

Index(['Year', 'Make', 'Model', 'Security', 'hardware_part_number'], dtype='object')

In [19]:
# Rename the columns to import to Supabase
df_chevrolet_renamed_columns = df_chevrolet_xlsx.rename(columns={'Year': 'year', 'Make': 'make', 'Model': 'model', 'Security': 'security'})

In [20]:
# Print the columns to see the results
df_chevrolet_renamed_columns.columns

Index(['year', 'make', 'model', 'security', 'hardware_part_number'], dtype='object')

In [48]:
# Download the csv file to import to Supabase
df_chevrolet_renamed_columns.to_csv("C:\Language_Projects\Language_Projects\Python\Flagship_1\Immo_Assistant.app\dataframe_csv_files\df_chevrolet_brands_hdw.csv", index=False)

<>:2: SyntaxWarning: invalid escape sequence '\L'
<>:2: SyntaxWarning: invalid escape sequence '\L'
C:\Users\Weverson Barbieri\AppData\Local\Temp\ipykernel_17444\3851937748.py:2: SyntaxWarning: invalid escape sequence '\L'
  df_chevrolet_renamed_columns.to_csv("C:\Language_Projects\Language_Projects\Python\Flagship_1\Immo_Assistant.app\dataframe_csv_files\df_chevrolet_brands_hdw.csv", index=False)


In [21]:
# Load the xlsx file containing the clonable hardware part numbers   
list_clonable_modules = pd.read_excel('C:\Language_Projects\Language_Projects\Python\Flagship_1\Immo_Assistant.app\data\List of Clonable Modules.xlsx')

<>:2: SyntaxWarning: invalid escape sequence '\L'
<>:2: SyntaxWarning: invalid escape sequence '\L'
C:\Users\Weverson Barbieri\AppData\Local\Temp\ipykernel_8552\1625161208.py:2: SyntaxWarning: invalid escape sequence '\L'
  list_clonable_modules = pd.read_excel('C:\Language_Projects\Language_Projects\Python\Flagship_1\Immo_Assistant.app\data\List of Clonable Modules.xlsx')


In [22]:
# Print the column names
list_clonable_modules.columns

Index(['Cloning Tools', 'ECM Hardware', 'cloning_status', 'ECU Brand', 'Year',
       'Year/Model/Engine Size/HP', 'Manufacturer', 'Vehicle Type', 'Notes'],
      dtype='object')

In [23]:
# Drop most columns and leave only 2 columns named: 'ECM Hardware' and 'cloning_status' and store in a variable
list_clonable_modules_updated = list_clonable_modules.drop(columns=['Cloning Tools', 'ECU Brand', 'Year', 'Year/Model/Engine Size/HP', 'Manufacturer', 'Vehicle Type', 'Notes'])

In [24]:
# Print the first rows on the dataframe updated
list_clonable_modules_updated.head()

,ECM Hardware,cloning_status
0,E83 (GM),Currently possible
1,E78 (GM),Currently possible
2,E87 (GM),Currently possible
3,E39 (GM),Currently possible
4,E39A (GM),Currently possible


In [25]:
# Filter the hardware pn# that can be cloned
df_list_clonable_modules_filtered = list_clonable_modules_updated[list_clonable_modules_updated['cloning_status'] == 'Currently possible']

In [26]:
# Print the columns
df_list_clonable_modules_filtered.columns

Index(['ECM Hardware', 'cloning_status'], dtype='object')

In [27]:
# Rename column ecm_hdw
df_list_clonable_modules_chevrolet = df_list_clonable_modules_filtered.rename(columns={'ECM Hardware': 'ecm_hardware'})

#### Data Processing and Database Import for Clonable Hardware Part Numbers
##### After cleaning the DataFrame containing the clonable hardware part numbers:
1. Export the data as a CSV file locally
2. Import it into the Supabase database

In [29]:
# Export the data as a CSV file locally
df_list_clonable_modules_chevrolet.to_csv('C:\Language_Projects\Language_Projects\Python\Flagship_1\Immo_Assistant.app\dataframe_csv_files\df_chevrolet_clonable_list.csv', index=False)

<>:2: SyntaxWarning: invalid escape sequence '\L'
<>:2: SyntaxWarning: invalid escape sequence '\L'
C:\Users\Weverson Barbieri\AppData\Local\Temp\ipykernel_17444\97026906.py:2: SyntaxWarning: invalid escape sequence '\L'
  df_list_clonable_modules_chevrolet.to_csv('C:\Language_Projects\Language_Projects\Python\Flagship_1\Immo_Assistant.app\dataframe_csv_files\df_chevrolet_clonable_list.csv', index=False)


In [28]:
# Call the function to get a cleared df after query from Supabase
df_chevrolet_clonable_hdw = query_table_from_supabaseto_df("chevrolet_clonable_table")

In [29]:
# Print the head
df_chevrolet_clonable_hdw.head(2)

,ecm_hardware,cloning_status
0,E83 (GM),Currently possible
1,E78 (GM),Currently possible


In [30]:
# Query all records from chevrolet_brands_hdw_table in Supabase and store the response
df_chevrolet = query_table_from_supabaseto_df('chevrolet_brands_hdw_table')

In [31]:
# Print the columns
df_chevrolet.columns

Index(['year', 'make', 'model', 'security', 'hardware_part_number'], dtype='object')

In [32]:
# Create a list to store the hardwares prefixes got from the clonable list
clonable_modules_list = []

# Iterate in the column 'ECM Hardware' to get their prefixes
for part_number in df_chevrolet_clonable_hdw['ecm_hardware']:
    ecm_hardware_prefix = part_number.split(' ')[0]
    clonable_modules_list.append(ecm_hardware_prefix)

In [33]:
# Print the last rows on the dataframe containing the hardware part number values
df_chevrolet.tail()

,year,make,model,security,hardware_part_number
657,2013,GMC,Savana,PK3+,E38-AK8
658,2013,GMC,Sierra,PK3+,E86-A24
659,2013,GMC,Yukon XL,PK3+,Information not available
660,2013,GMC,Yukon Denali,PK3+,E38-AK8
661,2013,GMC,Terrain,PK3+\nSmart Key,E39A-H98S


In [34]:
# Create a list to store hardware prefixes
hardware_prefix_list = []

# Iterate into the main dataframe to get the prefixes 
for hardware_pn in df_chevrolet['hardware_part_number']:
    hardware_prefix = hardware_pn.split('-')[0] # [0] indicates the first part of the list where the string needed is located, eg: 'E39', 'E38', etc.
    hardware_prefix_list.append(hardware_prefix)
    
# Add a new column contaning the prefixes
df_chevrolet['hardware_prefix'] = hardware_prefix_list

In [35]:
# Create a list to store the the string "Yes" or "No"
hardware_clonable_list = []
# Variable containing the string "Yes" to be used when a given part number is in the clonable_modules_list
yes_clonable = 'Yes'
# Variable containing the string "No" to be used when a given part number IS NOT in the clonable_modules_list
not_clonable = 'No'
# Loop in the 'hardware_prefix' column to check whether or not the part numbers are clonable 
for hardware_pn in df_chevrolet['hardware_prefix']:
    if hardware_pn in clonable_modules_list:
        hardware_clonable_list.append(yes_clonable)
    else:
        hardware_clonable_list.append(not_clonable)

# Create a new column after confirming the part numbers are clonable or not 
df_chevrolet['Is this hardware clonable?'] = hardware_clonable_list

In [112]:
# Drop column 'hardware_prefix' as it was used only to confirm the clonable hardwares
df_chevrolet_dropped_hardwarepre = df_chevrolet.drop(columns='hardware_prefix')

##### Data Cleaning Analysis
The security system descriptions require standardization to ensure data consistency. The following string patterns and special characters need to be removed:

##### Required Changes and Text Standardization
* Remove special characters
* Normalize security system descriptions
* Standardize naming conventions

1. Special Characters

* Remove '+' from system names (e.g., 'PK3+' → 'PK3')
* Remove '\n' line breaks
* Remove '*' symbols

2. System-Specific Strings
* Remove 'Smart Key' references
* Remove 'KA+' notation
* Remove model-specific notations:
* '(GT only)'
* '(Z24 only)'
* 'only' suffixes

In [140]:
# Show all security systems on Chevrolet brands
chevrolet_security_systems_list = df_chevrolet_dropped_hardwarepre['security'].value_counts()

In [113]:
# Show all makes under Chevrolet and use this list to guide when creating separate Dataframes
chevrolet_brands_list = df_chevrolet_dropped_hardwarepre['make'].unique()

In [135]:
# Print the list to be analysed
chevrolet_security_systems_list

security
PK3+                   195
PL2                    175
PK2                     64
PK3                     57
PK1                     53
PK3+\nSmart Key         38
PL3                     19
KA                      15
PL3/PK3+                10
EZ                       5
PL1                      4
PK3+/KA                  4
VATS                     4
Holden                   3
Toyota\nImmobilizer      2
Opel                     2
PK3/KA+                  2
Opt                      2
PL1 (GT\nonly*)          1
PL1(Z24\nonly*)          1
GTA\nonly                1
Name: count, dtype: int64

In [136]:
# Create a pattern
chevrolet_security_column_regex_pattern = r'\+|/KA|/|\n|\*|Smart\sKey|KA\+|\s*\(GT.*?\)|\s*\(Z24.*?\)|\s+only|\s{2,}'

In [137]:
# Replace uncessary data from column 'security' based on the pattern using regex
df_chevrolet_dropped_hardwarepre['security'] = (df_chevrolet_dropped_hardwarepre['security'].str.replace(chevrolet_security_column_regex_pattern, ' ', regex=True).str.strip())

In [138]:
# Print the data on security column to see the results
df_chevrolet_dropped_hardwarepre['security'].value_counts()

security
PK3                   296
PL2                   175
PK2                    64
PK1                    53
PL3                    19
KA                     15
PL3 PK3                10
PL1                     6
EZ                      5
VATS                    4
Holden                  3
Toyota Immobilizer      2
Opel                    2
Opt                     2
GTA                     1
Name: count, dtype: int64

In [139]:
# Print the head to see the results
df_chevrolet_dropped_hardwarepre.head(5)

,year,make,model,security,hardware_part_number,Is this hardware clonable?
0,1986,Chevrolet,Corvette,VATS,Information not available,No
1,1987,Chevrolet,Corvette,VATS,Information not available,No
2,1988,Chevrolet,Corvette,VATS,Information not available,No
3,1988,Pontiac,Firebird,GTA,Information not available,No
4,1989,Cadillac,Allante,PK1,Information not available,No


In [182]:
# Rename the last column
df_chevrolet_last_version = df_chevrolet_dropped_hardwarepre.rename(columns={'Is this hardware clonable?': 'is_this_hardware_clonable', 'hardware_part_number': 'hdw_pn'})

In [114]:
#  Creates a filtered DataFrame containing only records for a specific vehicle make.
def create_df_make(make, df):
    # Filter the dataframe based on the make given as str and the column 'make'
    df_make = df[df['make'] == make]

    # Return the df
    return df_make

In [183]:
# Create a dict to store all dfs from Chevrolet brands 
chevrolet_dfs = {}

# Iterate over the main df and column 'make'
for make in df_chevrolet_last_version['make']:
    # Filter and the create the df based on the make name
    df_make = df_chevrolet_last_version[df_chevrolet_last_version['make'] == make]
    # update the dict
    chevrolet_dfs.update({make: df_make})

### Data Export and Supabase Import Process

#### Overview
After processing the vehicle security system data, each manufacturer's DataFrame is exported to CSV files and prepared for Supabase database import. The data structure requires specific column naming conventions to maintain consistency and follow database best practices.

#### Column Standardization
Two main functions handle column renaming:

- rename_columns_ford_dfs(): For Ford/Mercury vehicles
- rename_columns_chevrolet_dfs(): For GM brand vehicles

These functions convert column names to snake_case format, which is:
1. More database-friendly
2. Follows SQL naming conventions
3. Ensures consistency across tables
4. Makes queries more readable

#### Supabase Implementation
##### The CSV files will be imported into separate tables in Supabase:

- Ford vehicles → ford_table
- Mercury vehicles → mercury_table
- Chevrolet vehicles → chevrolet_table
- Other GM brands → respective brand tables

This separation is necessary because:
- Each manufacturer has unique security system characteristics
- Different data structures (e.g. Ford's PATS vs GM's hardware cloning information)
- Enables more efficient queries and table management
- Maintains data organization for the Streamlit interface

In [159]:
# Print the chevrolet brands list
chevrolet_brands_list

array(['Chevrolet', 'Pontiac', 'Cadillac', 'Buick', 'Oldsmobile', 'GMC',
       'Saturn', 'Hummer'], dtype=object)

In [160]:
# Create the df with Chevrolet data
df_chevrolet = chevrolet_dfs['Chevrolet']

In [ ]:
# Export the Chevrolet df to csv file
df_chevrolet.to_csv("C:\Language_Projects\Language_Projects\Python\Flagship_1\Immo_Assistant.app\dataframe_csv_files\df_chevy_filtered.csv", index=False)

In [184]:
# Create the df with Pontiac data
df_pontiac = chevrolet_dfs['Pontiac']

In [187]:
df_pontiac

,year,make,model,security,hdw_pn,is_this_hardware_clonable
3,1988,Pontiac,Firebird,GTA,Information not available,No
9,1989,Pontiac,Firebird,PK1,Information not available,No
19,1990,Pontiac,Firebird,PK1,Information not available,No
31,1991,Pontiac,Firebird,PK1,Information not available,No
46,1992,Pontiac,Firebird,PK1,Information not available,No
...,...,...,...,...,...,...
514,2009,Pontiac,Vibe,Toyota Immobilizer,Information not available,No
554,2010,Pontiac,G3,Opt,Information not available,No
555,2010,Pontiac,G6,PK3,Information not available,No
556,2010,Pontiac,G8,PK3,Information not available,No


In [186]:
# Export the Pontiac df to csv file
df_pontiac.to_csv("C:\Language_Projects\Language_Projects\Python\Flagship_1\Immo_Assistant.app\dataframe_csv_files\df_pontiac.csv", index=False)

<>:2: SyntaxWarning: invalid escape sequence '\L'
<>:2: SyntaxWarning: invalid escape sequence '\L'
C:\Users\Weverson Barbieri\AppData\Local\Temp\ipykernel_8552\2472780175.py:2: SyntaxWarning: invalid escape sequence '\L'
  df_pontiac.to_csv("C:\Language_Projects\Language_Projects\Python\Flagship_1\Immo_Assistant.app\dataframe_csv_files\df_pontiac.csv", index=False)


In [162]:
# Create the df with Cadillac data
df_cadillac = chevrolet_dfs['Cadillac']

In [ ]:
# Export the Cadillac df to csv file
df_cadillac.to_csv("C:\Language_Projects\Language_Projects\Python\Flagship_1\Immo_Assistant.app\dataframe_csv_files\df_cadillac.csv", index=False)

In [163]:
# Create the df with Buick data
df_buick = chevrolet_dfs['Buick']

In [ ]:
# Export the Buick df to csv file
df_buick.to_csv("C:\Language_Projects\Language_Projects\Python\Flagship_1\Immo_Assistant.app\dataframe_csv_files\df_buick.csv", index=False)

In [164]:
# Create the df with Oldsmobile data
df_oldsmobile = chevrolet_dfs['Oldsmobile']

In [ ]:
# Export the Oldsmobile df to csv file
df_oldsmobile.to_csv("C:\Language_Projects\Language_Projects\Python\Flagship_1\Immo_Assistant.app\dataframe_csv_files\df_oldsmobile.csv", index=False)

In [165]:
# Create the df with GMC data
df_gmc = chevrolet_dfs['GMC']

In [ ]:
# Export the GMC df to csv file
df_gmc.to_csv('C:\Language_Projects\Language_Projects\Python\Flagship_1\Immo_Assistant.app\dataframe_csv_files\df_gmc.csv', index=False)

In [166]:
# Create the df with GMC data
df_saturn = chevrolet_dfs['Saturn']

In [ ]:
# Export the Saturn df to csv file
df_saturn.to_csv('C:\Language_Projects\Language_Projects\Python\Flagship_1\Immo_Assistant.app\dataframe_csv_files\df_saturn.csv', index=False)

In [167]:
# Create the df with Hummer data
df_hummer = chevrolet_dfs['Hummer']

In [ ]:
# Export the Hummer df to csv file
df_hummer.to_csv("C:\Language_Projects\Language_Projects\Python\Flagship_1\Immo_Assistant.app\dataframe_csv_files\df_hummer.csv", index=False)

In [116]:
# Function to rename the Ford dataframe columns 
def rename_columns_ford_dfs(df_make):
    # Create a map to rename the columns
    map_rename_columns_ford_makes = {
                                'Year': 'year',
                                'Make': 'make',
                                'Model': 'model',
                                'ParameterReset': 'parameter_reset',
                                'PATS Type': 'pats_type',
                                'PATS Module Location': 'pats_module_location'
                                }
    # Rename the columns using the map
    df_make_final_version = df_make.rename(columns=map_rename_columns_ford_makes)
    return df_make_final_version

In [117]:
# Function to rename the Chevrolet dataframe columns 
def rename_columns_chevrolet_dfs(df_make):
    # Create a map to rename the columns
    map_rename_columns_chevrolet_makes = {
                                        'Year': 'year', 
                                        'Make': 'make', 
                                        'Model': 'model', 
                                        'Security': 'security', 
                                        'Hardware Part Number': 'hdw_pn',
                                        'Is this hardware clonable?': 'is_this_hdw_clonable'
                                    }
    # Rename the columns using the map
    df_make_final_version = df_make.rename(columns=map_rename_columns_chevrolet_makes)
    return df_make_final_version